# Interactive Scatter Plot with hvPlot and Panel

Build an interactive scatter plot explorer using [hvPlot](https://hvplot.holoviz.org/) for visualization and [Panel](https://panel.holoviz.org/) for interactivity.

In [1]:
import panel as pn
import param
import hvplot.pandas  # noqa: adds .hvplot accessor to DataFrames

pn.extension(throttled=True)

## Load the Data

We use the [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/) dataset: 344 penguins, 4 numeric measurements, and 3 categorical columns (`species`, `island`, `sex`).

In [2]:
@pn.cache
def get_data():
    return hvplot.sampledata.penguins("pandas")

df = get_data()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


## A Static Scatter Plot

First, let's see what a basic hvPlot scatter looks like.

In [3]:
df.hvplot.scatter(
    x="bill_length_mm", y="flipper_length_mm",
    by="species", responsive=True, height=400,
)

:NdOverlay   [species]
   :Scatter   [bill_length_mm]   (flipper_length_mm)

## Make It Interactive with a Viewer Class

The `pn.viewable.Viewer` pattern uses the **declarative API**: all state lives in `param` parameters, the layout is created once in `__init__`, and `@param.depends` marks which methods to re-run when parameters change.

In [4]:
NUMERIC_COLS = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
CATEGORICAL_COLS = ["species", "island", "sex"]


class PenguinsExplorer(pn.viewable.Viewer):
    x_col = param.Selector(default="bill_length_mm", objects=NUMERIC_COLS, doc="X axis")
    y_col = param.Selector(default="flipper_length_mm", objects=NUMERIC_COLS, doc="Y axis")
    color_col = param.Selector(default="species", objects=CATEGORICAL_COLS, doc="Color by")

    def __init__(self, **params):
        super().__init__(**params)
        with pn.config.set(sizing_mode="stretch_width"):
            self._x_widget = pn.widgets.Select.from_param(self.param.x_col, name="X Axis")
            self._y_widget = pn.widgets.Select.from_param(self.param.y_col, name="Y Axis")
            self._color_widget = pn.widgets.Select.from_param(self.param.color_col, name="Color By")
            self._inputs = pn.Row(self._x_widget, self._y_widget, self._color_widget)
            self._plot_pane = pn.pane.HoloViews(self._create_scatter, sizing_mode="stretch_width")
            self._layout = pn.Column("# Penguins Explorer", self._inputs, self._plot_pane)

    @param.depends("x_col", "y_col", "color_col")
    def _create_scatter(self):
        data = get_data()
        return data.hvplot.scatter(
            x=self.x_col, y=self.y_col, by=self.color_col,
            responsive=True, height=400,
            title=f"{self.y_col} vs {self.x_col} (by {self.color_col})",
        )

    def __panel__(self):
        return self._layout

    @classmethod
    def create_app(cls, **params):
        instance = cls(**params)
        return pn.template.FastListTemplate(
            title="Penguins Explorer",
            sidebar=[instance._inputs],
            main=[instance._plot_pane],
            main_layout=None,
        )

In [5]:
explorer = PenguinsExplorer()
explorer

PenguinsExplorer(color_col='species', name='PenguinsExplorer00354', x_col='bill_length_mm', y_col='flipper_length_mm')

## Serving as a Standalone App

When served with `panel serve`, the cell below wraps the explorer in a `FastListTemplate` with a sidebar.

In [6]:
if pn.state.served:
    PenguinsExplorer.create_app().servable()

Run from the terminal:

```bash
panel serve example_notebooks/01_hvplot_scatter.ipynb --dev --show
```

## Copy-and-Adapt Checklist

To use this pattern with your own data:

1. Replace `get_data()` with your data loading function
2. Update `NUMERIC_COLS` and `CATEGORICAL_COLS` for your columns
3. Change `.hvplot.scatter(...)` to your preferred plot type (`.hvplot.line(...)`, `.hvplot.bar(...)`, etc.)
4. Add more `param.Selector` / `param.Integer` / etc. for additional widgets